In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib import animation, rc
from scipy import stats
import pandas as pd
import statistics
import seaborn as sns
import matplotlib.cm as cm
from tqdm.notebook import tqdm
def bind_dissociate(state, output_v, df_x, F):
    d_rate = (df_k_off * np.exp(df_x/df_lam)) #k_off(x)
    d_or = pd.DataFrame(np.random.random((N, n*loop)))
    d = d_or < d_rate.iloc[:N,:]
    b_or = pd.DataFrame(np.random.random((N, n*loop)))
    b = b_or < df_k_on.iloc[:N,:]
    #Determine binding and dissociation states stochastically
    state = ((state == True) & (d == False)) | ((state == False) & (b == True))
    n_on = [state.iloc[:j,i].sum() for i,j in enumerate(N_list)]
    n_off = [x1 - x2 for (x1, x2) in zip(N_list, n_on)]
    #Determine the speed by calculating the resistance from the distance traveled.
    df_delta_x = pd.DataFrame(index=range(N),columns=range(n*loop))
    df_delta_x.fillna(0, inplace=True)
    delta_x = [i * delta_t for i in output_v]
    for i in range(n*loop):
        df_delta_x[i]=delta_x[i]
    df_x=pd.DataFrame(np.where(state==True,df_x+df_delta_x,0))
    v = [F[i]/gamma_c[i] - beta[i] * df_x.iloc[:j,i].sum() for i,j in enumerate(N_list)]
    #[1-sp[i]*df_x.iloc[:j,i].sum()/F_0[i] for i,j in enumerate(N_list)]
    v = [i if i>0 else 0 for i in v]
    output_v = v
    F = [output_v[i]*gamma_c[i] + beta[i] * df_x.iloc[:j,i].sum() for i,j in enumerate(N_list)]
    output_F = [i if i>0 else 0 for i in F]
    return state, n_on, output_v, df_x, output_F

In [ ]:
L=5
for i in range(L):
    if i == 0:
        fig = plt.figure(figsize=(8*L,5))
    exec_command='ax'+str(i+1)+' = fig.add_subplot(1,'+str(L)+','+str(i+1)+')'
    exec(exec_command)
###############################################################
#parameters
colorlist = ["g", "r" , "b", "m", "c", "y", "k", ]
judge1,judge2=[],[]
T = 1000
delta_t = 5/T
n,loop = 10,3
vs  = 0.2
v_0,F_0 = [1 for i in range(n)]*loop,[1 for i in range(n)]*loop
sp_each = [0.25]*n*loop
sp = [sp_each[i] for i in range(n*loop)]
lam = [0.01 for i in range(n)]*loop
g = [1]*n*loop
gamma_c = [g[i] for i in range(n*loop)]
beta = [i / j for (i, j) in zip(sp, gamma_c)]
k_off = [0.03 for i in range(n)]*loop
#Choose which condition to change, N or Kd.
N_or_K = 'NandK'#'K'
if N_or_K == 'N': N_list,K_d = [500,1000,2000,3000,4000,5000,6000,8000,10000]*loop,[1 for i in range(n)]*loop
if N_or_K == 'K': N_list,K_d = [2000 for i in range(n)]*loop,[0.8, 1, 1.2, 1.25, 1.5]*loop
if N_or_K == 'NandK':
    K_d_each = [0.03, 0.5,10000]
    N_list,K_d = [1000,1000,1000,1000,1000,1000,1000,1000,1000,1000]*loop,[K_d_each[i//n] for i in range(n*loop)]
N = max(N_list)
k_on = [i / j for (i, j) in zip(k_off, K_d)]
#Create df to store parameters for each condition
df_K_d = pd.DataFrame(index=range(N),columns=range(n*loop))
df_sp = pd.DataFrame(index=range(N),columns=range(n*loop))
df_lam = pd.DataFrame(index=range(N),columns=range(n*loop))
df_k_off = pd.DataFrame(index=range(N),columns=range(n*loop))
df_K_d.fillna(0, inplace=True)
df_sp.fillna(0, inplace=True)
df_lam.fillna(0, inplace=True)
df_k_off.fillna(0, inplace=True)
for i in range(n*loop):
    df_K_d[i]=K_d[i]
    df_sp[i]=sp[i]
    df_lam[i]=lam[i]
    df_k_off[i]=k_off[i]
df_k_on = df_k_off/df_K_d
###############################################################
#Initial condition
v = [vs for i in range(n*loop)]
df_x=pd.DataFrame(index=range(N),columns=range(n*loop))
df_x.fillna(0, inplace=True)
b_or = pd.DataFrame(np.random.random((N, n*loop)))
b = b_or < df_k_on
state = b
n_on = [state.iloc[:j,i].sum() for i,j in enumerate(N_list)]
n_off = [x1 - x2 for (x1, x2) in zip(N_list, n_on)]
df_delta_x = pd.DataFrame(index=range(N),columns=range(n*loop))
df_delta_x.fillna(0, inplace=True)
delta_x = [i * delta_t for i in v]
for i in range(n*loop):
    df_delta_x[i]=delta_x[i]
df_x=pd.DataFrame(np.where(state==True,df_x+df_delta_x,0))
v = [F_0[i]/gamma_c[i] - beta[i] * df_x.iloc[:j,i].sum() for i,j in enumerate(N_list)]
output_v = [i if i>0 else 0 for i in v]
F = [output_v[i]*gamma_c[i] + beta[i] * df_x.iloc[:j,i].sum() for i,j in enumerate(N_list)]
output_F = [i if i>0 else 0 for i in F]
time=[]
for t in range(T):
    state, n_on, output_v, df_x, output_F = bind_dissociate(state, output_v, df_x, output_F)
    if t==0:
        output_x = [i * delta_t for i in output_v]
        df_n, df_v, df_dis = pd.DataFrame([n_on]),pd.DataFrame([output_v]),pd.DataFrame([output_x])
        df_nratio = pd.DataFrame([[j/N_list[i] for i,j in enumerate(n_on)]])
    else:
        output_x = [output_x[i]+j * delta_t for i,j in enumerate(output_v)]
        df_n, df_v, df_dis = pd.concat([df_n,pd.DataFrame([n_on])]).reset_index(drop=True),pd.concat([df_v,pd.DataFrame([output_v])]).reset_index(drop=True),pd.concat([df_dis,pd.DataFrame([output_x])]).reset_index(drop=True)
        df_nratio = pd.concat([df_nratio,pd.DataFrame([[j/N_list[i] for i,j in enumerate(n_on)]])]).reset_index(drop=True)
    time.append(t)#*delta_t
    somitu1=[i for i, x in enumerate(output_x) if 0.5< x <0.8]
    somitu2=[i for i, x in enumerate(output_x) if 0.8<= x]
    if somitu1:
        for h in somitu1:
            if h in judge1: pass
            else:
                N_list[h]=int(N_list[h]/5)
                N = max(N_list)
                judge1.append(h)
                #print(N_list)
    if somitu2:
        for h in somitu2:
            if h in judge2: pass
            else:
                N_list[h]=int(N_list[h]*5)
                N = max(N_list)
                judge2.append(h)
plt.rcParams["font.size"] = 20
for nn in range(n*loop):#
    ax2.set_ylim(0, 1.2)
    #ax3.set_ylim(0, 0.2)
    if nn/n==0:
        if int(nn//n)<n:
            ax4.plot(df_K_d.iloc[1,:].T.iloc[::n],df_nratio.iloc[int(T/5):,].mean().iloc[int(nn/n)::n])
        else:
            if nn>=n*(loop-1):
                ax1.plot(time, df_n.iloc[:,nn],label=f'{round(df_nratio.iloc[int(T/5):,].mean().iloc[nn], 2)}',c=colorlist[nn//n])
                ax2.plot(time, df_v.iloc[:,nn],label=f'{round(df_nratio.iloc[int(T/5):,].mean().iloc[nn], 2)}',c=colorlist[nn//n])
                ax3.plot(time, 0.2*df_dis.iloc[:,nn],label=f'{1/K_d[nn]}',c=colorlist[nn//n])
            else:
                ax1.plot(time, df_n.iloc[:,nn],c=colorlist[nn//n])
                ax2.plot(time, df_v.iloc[:,nn],c=colorlist[nn//n])
                ax3.plot(time, df_dis.iloc[:,nn],c=colorlist[nn//n])
            #print(N_list[nn],K_d[nn],sp[nn])
    else:
        if nn>=n*(loop-1):
            ax1.plot(time, df_n.iloc[:,nn],c=colorlist[nn//n])
            ax2.plot(time, df_v.iloc[:,nn],c=colorlist[nn//n])
            ax3.plot(time, 0.2*df_dis.iloc[:,nn],c=colorlist[nn//n])
        else:
            ax1.plot(time, df_n.iloc[:,nn],c=colorlist[nn//n])
            ax2.plot(time, df_v.iloc[:,nn],c=colorlist[nn//n])
            ax3.plot(time, df_dis.iloc[:,nn],c=colorlist[nn//n])
        #print(N_list[nn],K_d[nn],sp[nn])
#ax3.legend()
start = 0.5
end = 0.8
ax3.axhspan(start, end, color="gray", alpha=0.2)
fig.tight_layout()
plt.show()
plt.close()